In [2]:
import re

with open(r'D:\Programowanie\Projekty\chess_cheater_project\cheater_PGNs\white_cheater_PGN.txt') as f:
    pgn = f.read()

def moves_to_list(moves):
    moves_list = []
    for element in moves.split():
        if re.match('[a-zA-Z]+[1-9]|O-O|O-O-O', element) is not None:
            element = element.replace('?', '')
            element = element.replace('!', '')
            element = element.replace('+', '')

            moves_list.append(element)
    
    return moves_list

moves = moves_to_list(pgn)

In [3]:
len(moves)

113

In [4]:
import chess
import numpy as np

def save_board_state_after_move(moves):
    board = chess.Board()
    FEN_list = []
    try:
        for idx, move in enumerate(moves):
            board.push_san(move)
            FEN_list.append(board.fen())
        return FEN_list
    except Exception as e:
        print(e)
        print(board)
        return np.nan

fens = save_board_state_after_move(moves)

In [5]:
import pandas as pd
import numpy as np

next_fens = fens[1:]
next_fens.append(np.nan)

data = {'fens': fens, 'next_fens': next_fens}

input_data = pd.DataFrame(data=data)
input_data.dropna(inplace=True)

In [6]:
piece_to_token   = {'r' :        1,
                   'n' :        2,
                   'b' :        3,
                   'q' :        4,
                   'k' :        5,
                   'p' :        6,
                   'R' :        7,
                   'N' :        8,
                   'B' :        9,
                   'Q' :        10,
                   'K' :        11,
                   'P' :        12,
                   'empty' :    0}

def fen_to_network_input_tokenized(fen):
    encoded_board = []
    fen = fen.split()[0]
    for char in fen:
        if char in piece_to_token.keys():
            encoded_board.append(piece_to_token[char])
        elif char != '/':
            for _ in range(int(char)):
                encoded_board.append(piece_to_token['empty'])
    return encoded_board

In [7]:
input_data['FEN_tokenized'] = input_data['fens'].apply(fen_to_network_input_tokenized)
input_data['next_FEN_tokenized'] = input_data['next_fens'].apply(fen_to_network_input_tokenized)


In [8]:
input_data['input'] = input_data['FEN_tokenized'] + input_data['next_FEN_tokenized']

In [9]:
import torch

X = torch.tensor(input_data['input'],dtype=torch.int)

In [10]:
from model_lstm import CheaterClassifier

model = CheaterClassifier()

model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [11]:
X.shape

torch.Size([112, 128])

In [12]:
model.eval()
outputs = model(X)

In [13]:
counter_white=0
counter_black=0
mean_proba_white = 0
mean_proba_black = 0
white = []
black = []
for idx, output in enumerate(outputs):
    if idx % 2 == 0:
        mean_proba_white += output
    else:
        mean_proba_black += output
    if output >= .5:
        if idx % 2 == 0:
            white.append((idx, output))
            counter_white+=1
        else:
            black.append((idx, output))
            counter_black+=1

mean_proba_white = mean_proba_white/len(outputs)
mean_proba_black = mean_proba_black/len(outputs)
print('Mean proba white: ', mean_proba_white)

print('Mean proba black: ', mean_proba_black)
print('white', counter_white)
print('black', counter_black)
print('##########white############')
for w in white:
    print(w)
print('##########black############')
for b in black:
    print(b)

Mean proba white:  tensor([0.1209], grad_fn=<DivBackward0>)
Mean proba black:  tensor([0.1188], grad_fn=<DivBackward0>)
white 9
black 9
##########white############
(52, tensor([0.5319], grad_fn=<UnbindBackward0>))
(54, tensor([0.5600], grad_fn=<UnbindBackward0>))
(56, tensor([0.5760], grad_fn=<UnbindBackward0>))
(58, tensor([0.5823], grad_fn=<UnbindBackward0>))
(60, tensor([0.5840], grad_fn=<UnbindBackward0>))
(62, tensor([0.5737], grad_fn=<UnbindBackward0>))
(64, tensor([0.5574], grad_fn=<UnbindBackward0>))
(66, tensor([0.5353], grad_fn=<UnbindBackward0>))
(68, tensor([0.5078], grad_fn=<UnbindBackward0>))
##########black############
(51, tensor([0.5104], grad_fn=<UnbindBackward0>))
(53, tensor([0.5482], grad_fn=<UnbindBackward0>))
(55, tensor([0.5700], grad_fn=<UnbindBackward0>))
(57, tensor([0.5795], grad_fn=<UnbindBackward0>))
(59, tensor([0.5833], grad_fn=<UnbindBackward0>))
(61, tensor([0.5813], grad_fn=<UnbindBackward0>))
(63, tensor([0.5665], grad_fn=<UnbindBackward0>))
(65, ten